#### This notebook prepares the input data to be consumed by the Neural Network.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Run the cell below to extract the edgelist from .mtx file (the way Node2Vec expects the input). We will iterate through all the mtx files and create a corresponding .edgelist file

In [29]:
import os

data_files = os.listdir('../data')
for file in data_files:
    if file.endswith('.mtx'):
        file_name = file.replace('.mtx', '')
        file_edgelist = file_name+'.edgelist'
        if not file_edgelist in data_files:
            lines = None
            with open('../data/'+file) as file_mtx:
                lines = file_mtx.readlines()
            with open('../data/'+file_edgelist, 'w') as file_edgelist:
                file_edgelist.writelines(lines[2:])
                print(file_edgelist, 'created')

Now that we have extracted edgelists from .mtx files using above cell, let's generate node embeddings (node2vec). For that I will use the code that the author's have shared on there [Github](https://github.com/aditya-grover/node2vec). But first we have to convert the script from python2 to python3 and replace "import node2vec" with "import node2vec3".

In [3]:
# ! 2to3 -w './node2vec'  # didn't work. No such file or directory error
# os.listdir('./node2vec')  # while this works!
# so converted main.py and node2vec.py using online translators

graph_name = 'socfb-American75'

For generating embeddings use the parameters used in the "shortest path distance" paper.

In [6]:
%%time
import os

if not os.path.exists('../data/emb'):
    os.makedirs('../data/emb')
# ! python node2vec/main3.py --help
if graph_name == 'socfb-American75':
    ! python node2vec/main3.py --input ../data/socfb-American75.edgelist --output ../data/emb/socfb-American75.emd
    print('embedding saved at', '../data/emb/socfb-American75.emd')
else:
    ! python node2vec/main3.py --input ../data/socfb-OR.edgelist --output ../data/emb/socfb-OR.emd
    print('embedding saved at', '../data/emb/socfb-OR.emd')

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
embedding saved at ../data/emb/socfb-American75.emd
Wall time: 3min 2s


The Graph class has a <i>naive</i> implementation of Dijkstra's Algorithm to calculate distance of all the nodes from a specified source node. It is slow but since we need to run it for landmarks (number of landmarks << number of nodes) only I will go ahead with this. 

In [1]:
from graph_proc import Graph
from logger import Logger

logger = Logger('../outputs/logs', 'log_')
graph = Graph('../data/'+graph_name+'.mtx', logger)
save_path = graph.process_landmarks()

100%|██████████| 150/150 [37:05<00:00, 14.83s/it]save path: ../outputs/distance_map_1588792161.8904061.pickle



#### Calculate landmark distances using Networkx library, instead of shitty own implementation

In [7]:
import networkx as nx
import numpy as np

np.random.seed(999)
edgelist_path = '../data/'+graph_name+'.edgelist'
graph = nx.read_edgelist(edgelist_path, nodetype=int)

In [8]:
from tqdm.auto import tqdm
import pickle
import time

nodes = list(graph.nodes)  # [int(i) for i in list(graph.nodes)]
landmarks = np.random.randint(1, len(nodes), 150)

distance_map = {}
distances = np.zeros((len(nodes), ))

for landmark in tqdm(landmarks):
    distances[:] = np.inf
    node_dists = nx.shortest_path_length(graph, landmark)
    for key, value in node_dists.items():
        distances[key-1] = value
    distance_map[landmark] = distances.copy()  # copy because array is re-init on loop start

save_path = '../outputs/distance_map_'+graph_name+'_'+str(time.time())+'.pickle'
pickle.dump(distance_map, open(save_path, 'wb'))
print('distance_map saved at', save_path)

100%|██████████| 150/150 [00:22<00:00,  6.74it/s]distance_map saved at ../outputs/distance_map_socfb-American75_1590779864.8981035.pickle



The result is saved in a pickle file (dict) to analyse. As you can see below only isolated (disconnected from source) nodes are left out, which form a cycle with another node (isolated cycles). Same set of isolated nodes are found for all of the landmarks. So we can ignore them.

In [9]:
import pickle
import numpy as np
from scipy import io

mtx_path = '../data/'+graph_name+'.mtx'
mat_csr = io.mmread(mtx_path).tocsr()
distance_map = pickle.load(open(save_path, 'rb'))
keys = list(distance_map.keys())
count = 0
for key in keys:
    l = distance_map[key]
    hitlist = np.where(l==np.inf)[0]
    # print('Number of isolated keys for source-{} is {}'.format(key, len(hitlist)))
    if(len(hitlist) > 0):
        count += 1
    # for i in hitlist:
    #     print(i, '--', np.where(mat_csr[i].toarray()[0]>0)[0])
    # if(len(hitlist)>0):
    #     break
print('Number of sources for which any isolated nodes found are', count)

Number of sources for which any isolated nodes found are 149


All cells before this had to be run once to process the graph and save results to save time. Now we have to read the distance map and embeddings to form training data.

In [4]:
import numpy as np
import sys
import pickle

graph_name = 'socfb-American75'
save_path = '../outputs/distance_map_'+graph_name+'_1590779864.8981035.pickle'
distance_map = pickle.load(open(save_path, 'rb'))
emd_path = '../data/emb/'+graph_name+'.emd'
emd_map = {}
with open(emd_path, 'r') as file:
    lines = file.readlines()
    for line in lines[1:]:
        temp = line.split(' ')
        emd_map[np.int(temp[0])] = np.array(temp[1:], dtype=np.float)
print('size of emd_map:', sys.getsizeof(emd_map)/1024/1024,'MB')
print('size of distance_map:', sys.getsizeof(distance_map)/1024/1024,'MB')

size of emd_map: 0.28133392333984375 MB
size of distance_map: 0.00447845458984375 MB


In [5]:
from tqdm.auto import tqdm

emd_dist_pair = []
for landmark in tqdm(list(distance_map.keys())):
    node_distances = distance_map[landmark]
    emd_dist_pair.extend([((emd_map[node]+emd_map[landmark])/2, distance) for node, distance in enumerate(node_distances, 1) if node != landmark and distance != np.inf])

print('length of embedding-distance pairs', len(emd_dist_pair))

100%|██████████| 149/149 [00:03<00:00, 44.43it/s]length of embedding-distance pairs 948981



In [6]:
import sys

x = np.zeros((len(emd_dist_pair), len(emd_dist_pair[0][0])))
y = np.zeros((len(emd_dist_pair),))

for i, tup in enumerate(tqdm(emd_dist_pair)):
    x[i] = tup[0]
    y[i] = tup[1]
print("Shape of x={} and y={}".format(x.shape, y.shape))
print('size of x={} MB and y={} MB'.format(sys.getsizeof(x)/1024/1024, sys.getsizeof(y)/1024/1024))

100%|██████████| 948981/948981 [00:01<00:00, 637299.64it/s]Shape of x=(948981, 128) and y=(948981,)
size of x=926.7393646240234 MB and y=7.240242004394531 MB



In [7]:
print('Range of distance values', np.min(y), np.max(y))
print('frequency of each distance value', np.unique(y, return_counts=True))

Range of distance values 1.0 8.0
frequency of each distance value (array([1., 2., 3., 4., 5., 6., 7., 8.]), array([ 10174, 315781, 511818, 100163,  10163,    845,     36,      1],
      dtype=int64))


A sanity check...

In [8]:
num_neg_dist = 0
distances_ = []
for i, landmark in enumerate(distance_map.keys()):
    distances_.extend(distance_map[landmark])
print('number of negative distances', np.sum(np.array(distances_) < 0))
print('number of inf distances', np.sum(np.array(distances_)==np.inf))

number of negative distances 0
number of inf distances 2384


Since the data takes up a lot of space, let's convert the datatype of x and y. In case you are worried about the precision loss, I think you can save the converted data into separate ndarray(x1), and try "np.mean(np.abs(x-x1))". For this data it was very small (2.7954226433144966e-09),so ignoring it. And in our case graphs are unweighted, so distance would be integer always.

In [9]:
x = x.astype('float32')
y = y.astype('int')
print('size of x={} MB and y={} MB'.format(sys.getsizeof(x)/1024/1024, sys.getsizeof(y)/1024/1024))

size of x=463.36973571777344 MB and y=3.620166778564453 MB


Since we saw above that distance value occurences are heavily skewed, I decided to do a stratified train-test split. But since distance '8' has only one occurence I am dropping it, else train_test_split throws error-- "The least populated class in y has only 1 member, which is too few"

In [10]:
prev_len = x.shape[0]
mask = y!=8
x = x[mask]
y = y[mask]
print('{} rows deleted'.format(prev_len-x.shape[0]))

1 rows deleted


Now let's split the data into training, validation and test datasets.

In [11]:
from sklearn.model_selection import train_test_split
import torch

seed_random = 9999
np.random.seed(seed_random)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_random)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, train_size=0.75, random_state=seed_random, shuffle=True, stratify=y)
x_train, x_cv, y_train, y_cv = train_test_split(x_train, y_train, test_size=0.2, train_size=0.8, random_state=seed_random, shuffle=True, stratify=y_train)

print('shapes of train, validation, test data', x_train.shape, y_train.shape, x_cv.shape, y_cv.shape, x_test.shape, y_test.shape)

shapes of train, validation, test data (569388, 128) (569388,) (142347, 128) (142347,) (237245, 128) (237245,)


As discribed in point-2 in fun.ipyb, the distance values are heavily skewed. So oversampling under-represented samples. But here is the challenge-- how do you oversample a regression dataset? Didn't find any good existing implementations. But in this cases values are all natural numbers and these might as well be classes! So using SMOTE. 
Note: I am not using classification models because distance might be outside the range of [[1-7]]

In [12]:
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.preprocessing import StandardScaler

# TODO try standardization and no normalization also and compare result

scaler = StandardScaler()
# scaler = MinMaxScaler(feature_range=(0, 1))
x_train = scaler.fit_transform(x_train)
x_cv = scaler.transform(x_cv)
x_test = scaler.transform(x_test)

pickle.dump((x_train, y_train), open('../outputs/train_xy_no_sampling_stdScale.pk', 'wb'))
pickle.dump((x_cv, y_cv), open('../outputs/val_xy_no_sampling_stdScale.pk', 'wb'))
pickle.dump((x_test, y_test), open('../outputs/test_xy_no_sampling_stdScale.pk', 'wb'))